# Gmarket 의류 데이터 수집

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os

## 상품별 코드

- 남성 의류 : subGroupCode=S162, largeCategoryCode=100000104
    - 셔츠 / 남방 : 200002680
    - 티셔츠 : 200002681
    - 패딩 / 점퍼 : 200002684
    - 자켓 / 코트 : 200002682
    - 팬츠 : 200002687
    
- 여성 의류 : subGroupCode=S161, largeCategoryCode=100000103
    - 블라우스 / 셔츠 : 200002670
    - 티셔츠 : 200002669
    - 스커트 : 200002674
    - 바지 / 청바지 : 200002673
    - 원피스 : 200002671

In [2]:
man_subGroupCode = 'S162'
woman_subGroupCode = 'S161'

man_largeCategoryCode = 100000104
woman_largeCategoryCode = 100000103

man_product_code = {'shirts': 200002680, 'tshirts' : 200002681, 'jumper' : 200002684, 'jacket' : 200002682, 'pants' : 200002687}
woman_product_code = {'shirts': 200002670, 'tshirts' : 200002669, 'skirts' : 200002674, 'pants' : 200002673, 'onepiece' : 200002671}

datas = []

In [3]:
def gmarket_crawler(sex, product_code, subcode, largecode):
    global cnt
    
    for k, v in product_code.items():
        url = f'https://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G01&subGroupCode={subcode}&largeCategoryCode={largecode}&mediumCategoryCode={v}'
        
        response = requests.get(url)
        dom = BeautifulSoup(response.text, 'html.parser')
        elements = dom.select('#gBestWrap > div > div:nth-child(5) > div > ul > li')
        element = elements[0:21]
        
        for elm in element:
            datas.append({
                'id' : elm.select_one('img.lazy').get('data-original')[22:32],
                'sex' : sex,
                'category' : k,
                'title' : elm.select_one('.itemname').text,
                'price' : elm.select_one('.s-price > strong > span').text,
            })

In [4]:
gmarket_crawler('m', man_product_code, man_subGroupCode, man_largeCategoryCode)

In [5]:
gmarket_crawler('w', woman_product_code,  woman_subGroupCode, woman_largeCategoryCode)

In [6]:
datas

[{'id': '2812004427',
  'sex': 'm',
  'category': 'shirts',
  'title': '[앤드지 바이 지오지아] (신세계경기점)앤드지[ANDZ] 히든 릴렉스핏 남방 4종 택1 (BED1WC1101)',
  'price': '13,100원'},
 {'id': '1644625234',
  'sex': 'm',
  'category': 'shirts',
  'title': '[인디안] S/S 베이직 인기아이템 100종 ~ 최대95% +10%쿠폰',
  'price': '36,400원'},
 {'id': '2499726390',
  'sex': 'm',
  'category': 'shirts',
  'title': '[올젠] (롯데백화점)올젠 22SS 7종택1 링클프리 캐쥬얼 정장 베이직 셔츠 (ZPC5WD1151)',
  'price': '23,700원'},
 {'id': '2601482587',
  'sex': 'm',
  'category': 'shirts',
  'title': '[듀퐁] 갤러리아 듀퐁긴팔셔츠 기본 베스트상품 16종 택1',
  'price': '42,350원'},
 {'id': '1784354813',
  'sex': 'm',
  'category': 'shirts',
  'title': '[예작] 갤러리아 두장구매시양말증정23년직장인들이좋아하는셔츠모음전 YJ3SBS15',
  'price': '24,850원'},
 {'id': '1841014761',
  'sex': 'm',
  'category': 'shirts',
  'title': '[예작] 갤러리아 두장구매시양말증정(슬림핏일반핏)화이트모음전긴팔셔츠 YJ3S',
  'price': '26,600원'},
 {'id': '1483640883',
  'sex': 'm',
  'category': 'shirts',
  'title': '[란체티] (신세계경기점)화이트 모달 솔리드 슬림핏 긴소매셔츠 LZRP504WH',
  'price': '37,400

In [7]:
df = pd.DataFrame(datas)

In [8]:
df.head()

,id,sex,category,title,price
0,2812004427,m,shirts,[앤드지 바이 지오지아] (신세계경기점)앤드지[ANDZ] 히든 릴렉스핏 남방 4종 ...,"13,100원"
1,1644625234,m,shirts,[인디안] S/S 베이직 인기아이템 100종 ~ 최대95% +10%쿠폰,"36,400원"
2,2499726390,m,shirts,[올젠] (롯데백화점)올젠 22SS 7종택1 링클프리 캐쥬얼 정장 베이직 셔츠 (Z...,"23,700원"
3,2601482587,m,shirts,[듀퐁] 갤러리아 듀퐁긴팔셔츠 기본 베스트상품 16종 택1,"42,350원"
4,1784354813,m,shirts,[예작] 갤러리아 두장구매시양말증정23년직장인들이좋아하는셔츠모음전 YJ3SBS15,"24,850원"


In [9]:
df.tail()

,id,sex,category,title,price
205,2811425202,w,onepiece,[포커스] [대구백화점] [4CUS]봄 원피스/재킷 인기 BEST 14종,"27,000원"
206,2755567586,w,onepiece,[르샵] (현대Hmall)[애즈마들린] 핀턱 플레어 5부 롱 원피스 (AM3OP004),"13,600원"
207,2766217904,w,onepiece,[포커스] [대구백화점] [4CUS]스트링 데님 후드 롱 원피스 (FS3A7OP01...,"59,410원"
208,2567790112,w,onepiece,[듀엘] (신세계경기점)[듀엘]니트 배색 물결 레이스 원피스 D223MSE009,"259,370원"
209,2540735403,w,onepiece,[더 아이잗] 더아이잗 빅카라 버튼원피스 IM6N0OP77,"72,980원"


In [10]:
df['price'] = df['price'].str.replace('원', '')
df['price'] = df['price'].str.replace(',', '')
df['price'] = pd.to_numeric(df['price'])
df['id'] = pd.to_numeric(df['id'])

In [11]:
df.head()

,id,sex,category,title,price
0,2812004427,m,shirts,[앤드지 바이 지오지아] (신세계경기점)앤드지[ANDZ] 히든 릴렉스핏 남방 4종 ...,13100
1,1644625234,m,shirts,[인디안] S/S 베이직 인기아이템 100종 ~ 최대95% +10%쿠폰,36400
2,2499726390,m,shirts,[올젠] (롯데백화점)올젠 22SS 7종택1 링클프리 캐쥬얼 정장 베이직 셔츠 (Z...,23700
3,2601482587,m,shirts,[듀퐁] 갤러리아 듀퐁긴팔셔츠 기본 베스트상품 16종 택1,42350
4,1784354813,m,shirts,[예작] 갤러리아 두장구매시양말증정23년직장인들이좋아하는셔츠모음전 YJ3SBS15,24850


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        210 non-null    int64 
 1   sex       210 non-null    object
 2   category  210 non-null    object
 3   title     210 non-null    object
 4   price     210 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 8.3+ KB


In [ ]:
!pip install sqlalchemy

In [ ]:
!pip install pymysql

In [13]:
from sqlalchemy import create_engine

In [20]:
engine = create_engine("mysql+pymysql://:---"+"----"+"@localhost:3306/vanilla?charset=utf8", encoding='utf-8')

In [21]:
conn = engine.connect()

In [24]:
df.to_sql(name='product', con=engine, if_exists='append', index=False)

210